In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r'./owid-2022-clean.csv')

In [4]:
# Drop irrelevant fields
print(df.columns.shape)
df.drop(["location", "date", "total_deaths_per_million", "total_cases_per_million", "population", "tests_units"], axis=1, inplace=True)

print(df.columns.shape)

(31,)
(25,)


At this stage, 2 different methods of filling mising data will be tested: dropping columns with too many missing values and making educated guesses

In [5]:
# Create a copy of df
dfill =df.copy()

# Method 1: 

In [6]:
indicator_cols = pd.get_dummies(df[["iso_code", "continent"]])
df = pd.concat([df, indicator_cols], axis=1)
df.drop(["iso_code", "continent"], axis=1, inplace=True)

If more than half values of columns are unknown then drop it

In [7]:
# drop cols with too much missing values
for col_name, col_data in df.iteritems():
    if col_data.isna().sum() > len(col_data)//2:
        df.drop(col_name, inplace=True, axis=1)
        print(f"Drop {col_name}")

Drop people_vaccinated_per_hundred
Drop people_fully_vaccinated_per_hundred
Drop total_boosters_per_hundred
Drop handwashing_facilities


There are 4 features are dropped above. However, if a country has data for people_vaccinated_per_hundred and people_fully_vaccinated_per_hundred then it often has all 7 values in these 2 fields while others have no value in these. Therefore, they can be a good indicator for the prediction that we can perform later in certain coutries.  

In [8]:
df

,new_cases_per_million,new_deaths_per_million,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,stringency_index,population_density,median_age,aged_65_older,aged_70_older,...,iso_code_YEM,iso_code_ZAF,iso_code_ZMB,iso_code_ZWE,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America
0,15.790,0.075,NaN,NaN,NaN,19.44,54.422,18.6,2.581,1.337,...,0,0,0,0,0,1,0,0,0,0
1,15.941,0.025,NaN,NaN,NaN,19.44,54.422,18.6,2.581,1.337,...,0,0,0,0,0,1,0,0,0,0
2,13.480,0.050,NaN,NaN,NaN,19.44,54.422,18.6,2.581,1.337,...,0,0,0,0,0,1,0,0,0,0
3,15.840,0.000,NaN,NaN,NaN,19.44,54.422,18.6,2.581,1.337,...,0,0,0,0,0,1,0,0,0,0
4,8.861,0.126,0.033,0.4855,2.1,19.44,54.422,18.6,2.581,1.337,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,10.469,0.331,1.463,0.0070,143.9,50.00,42.729,19.6,2.822,1.882,...,0,0,0,1,1,0,0,0,0,0
1319,0.000,0.000,1.457,0.0054,183.9,50.00,42.729,19.6,2.822,1.882,...,0,0,0,1,1,0,0,0,0,0
1320,15.372,0.331,1.462,0.0064,156.4,50.00,42.729,19.6,2.822,1.882,...,0,0,0,1,1,0,0,0,0,0
1321,0.000,0.000,1.477,0.0060,165.6,50.00,42.729,19.6,2.822,1.882,...,0,0,0,1,1,0,0,0,0,0


#Method 2

In [10]:
dfc = dfill.groupby("continent")
dfm = dfc.mean().reset_index()
dfm